# Load and query data: `db.load`, `db.query`

In [ ]:
import lamindb as ln

ln.nb.header()

## Load

You can load dobjects into memory via `db.load`, for instance:

In [ ]:
dobject = ln.db.query.dobject(name="iris")[0]

df = ln.db.load(dobject)

In [ ]:
df.head()

During your analysis, you notice something strange, and you'd like to understand where the data came from.

Any dobject has a data source in form of a `dtransform_id`.

In [ ]:
dobject.dtransform_id

## Query

For simple frequent queries, LaminDB has a high-level query API based on SQLModel.[^sqlmodel]

[^sqlmodel]: We'll also show how to use SQLModel for arbitrarily complicated queries.

### Simple table queries

You can query any entity table in the database by its fields, for instance:

In [ ]:
ln.db.query.dobject(file_suffix=".feather")

The high-level query API by default returns a list of results:

In [ ]:
ln.db.query.species(taxon_id=10090)

You can set `return_df=True` to return a `DataFrame` instead.

In [ ]:
ln.db.query.species(taxon_id=10090, return_df=True)

If no parameters are passed, all entries in the table will be returns.

You can 

In [ ]:
ln.db.query.readout()

### Query data objects by linked entities

We also offer querying data objects by fields that are not present in the `dobject` table, but linked tables.

You can do this through providing a `entity_name` (name of the entity table), and a dictionary of query values.

In [ ]:
ln.db.query.dobject(entity_name="gene", entity_kwargs={"symbol": "Actg1"})

In [ ]:
ln.db.query.dobject(entity_name="gene", entity_kwargs={"ncbi_gene_id": 66722})

### Comprehensive SQL queries:

You can construct arbitrarily complicated SQL queries, and that we just saw queries for the primary key of an entity in the core schema module. More on this later.

Let's query for a data source:

In [ ]:
with ln.db.session() as session:
    dtransform = session.get(ln.schema.core.dtransform, dobject.dtransform_id)

Inspecting the result we see that the dobject originates from a Jupyter Notebook.

In [ ]:
dtransform

In [ ]:
with ln.db.session() as session:
    jupynb = session.get(
        ln.schema.core.jupynb,
        (dtransform.jupynb_id, dtransform.jupynb_v),  # it's version "1" see jupynb_v
    )

In [ ]:
jupynb

Now we found the person who last edited the notebook!

In [ ]:
with ln.db.session() as session:
    user = session.get(ln.schema.core.user, jupynb.user_id)

In [ ]:
user